### Write your own Smith-Waterman alignment script
- The script should take as input a tab delineated scoring matrix file, and two fasta formatted sequence text files.
- The script should produce as output a text file containing an optimized fasta formatted alignment based on the algorithm and input scoring matrix..

In [2]:
## Let's import os
import os
print(os.getcwd()) # this will get the current working directory

/home/user


In [3]:
# lets's  open the provided Fasta file ATU76697.fasta to have a look 
seq1 = open("ATU76697.fasta", 'r')
print(seq1.read())
seq1.close()

>gi|1790923|gb|U76697.1|ATU76697 Arabidopsis thaliana transposon-like element Limpet1, genomic sequence
GGGAATGTTCCAATATAAGACACTTTAAACGTAAGTTTAGACAATATAGACACTTTCCAAGTTAGAGGCA
CTTTTCCTTCTTTTTGAAGGAAAACTTGACTTTTATACCTCTTAACTAAACAATCGAAAACAATAACTAA
ATATATATCTTAACCAAACAATTAAAAAAATAAAAGAATTTAGATACGTAGTTATTAATATAGACCATTA
GATTGAAAAATAAAAATTAAGATCTATGGCTGAGATTAAAGACAATAAATGGATTAATTTTTTGATGTTA
AAATCTGATTAGAAAAAGGTATTTCTCTTCGTCTCTAAAACTAAATCTCTCTCTCTAAAAAAACAATCGT
TTCTCCCTTTCTCCTTCCTGAAGATCGTTTTTTCATAAATCCATAGTAGTTTAAAAACGAAGCAGAGAGA
TGTTGAAAATCGTTTCTCATGAAATTAATCGATTATTCTCTGTGAAGTTCTTTAATCCACACAACTTTCC
TCATGAACATGATAATAGTAGTAAATGGAGGTTTTTCCTATGGTTACTCTAGACGAAGGAGGATCTCCTT
GTGTTGGACAGGTTTGTGATTTCTTTCCATGGATTAAAAAAATTTGATTGTTTGTTTATGATGAACGATT
CTTTGGCTACGGAAGAGTGTCATGGAGTTCTGGCGAATTCTTTGGCTATGTTTGGTGATTTCGTTTTTAA
TCAAGTTGGGAATCAATAGGAAACAACTAAGCATACAACATAGATTAGAAGAGATATCAAGATGGATCTA
ATTTAAGTAAGATTTGGCGACTAATTCTAGATGATTAGGGTTATTTGTGATTTATTACAAGGCATTTGTG
TTCTCATTGATTTGGCGAGTAATTCTGTATGACTAGGGTTATTT

In [4]:
# read the sequence from a FASTA file ATU76697.fasta
def extract__transposonlikeelementsequence_from_fasta(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        sequence = ''.join(line.strip() for line in lines if not line.startswith('>'))
    return sequence

# Get the first sequence
file_path = "ATU76697.fasta"
seq1 = extract__transposonlikeelementsequence_from_fasta(file_path)
print("ATU76697 Sequence is:", seq1)
print("The length of the ATU76697 Sequence is:", len(seq1))

ATU76697 Sequence is: GGGAATGTTCCAATATAAGACACTTTAAACGTAAGTTTAGACAATATAGACACTTTCCAAGTTAGAGGCACTTTTCCTTCTTTTTGAAGGAAAACTTGACTTTTATACCTCTTAACTAAACAATCGAAAACAATAACTAAATATATATCTTAACCAAACAATTAAAAAAATAAAAGAATTTAGATACGTAGTTATTAATATAGACCATTAGATTGAAAAATAAAAATTAAGATCTATGGCTGAGATTAAAGACAATAAATGGATTAATTTTTTGATGTTAAAATCTGATTAGAAAAAGGTATTTCTCTTCGTCTCTAAAACTAAATCTCTCTCTCTAAAAAAACAATCGTTTCTCCCTTTCTCCTTCCTGAAGATCGTTTTTTCATAAATCCATAGTAGTTTAAAAACGAAGCAGAGAGATGTTGAAAATCGTTTCTCATGAAATTAATCGATTATTCTCTGTGAAGTTCTTTAATCCACACAACTTTCCTCATGAACATGATAATAGTAGTAAATGGAGGTTTTTCCTATGGTTACTCTAGACGAAGGAGGATCTCCTTGTGTTGGACAGGTTTGTGATTTCTTTCCATGGATTAAAAAAATTTGATTGTTTGTTTATGATGAACGATTCTTTGGCTACGGAAGAGTGTCATGGAGTTCTGGCGAATTCTTTGGCTATGTTTGGTGATTTCGTTTTTAATCAAGTTGGGAATCAATAGGAAACAACTAAGCATACAACATAGATTAGAAGAGATATCAAGATGGATCTAATTTAAGTAAGATTTGGCGACTAATTCTAGATGATTAGGGTTATTTGTGATTTATTACAAGGCATTTGTGTTCTCATTGATTTGGCGAGTAATTCTGTATGACTAGGGTTATTTGTGTTTTCTTAAAAAGAATTTGTGTTCTTGTTGAAATCTTGTTCATTGGAATTATTTGTGTTTGGTAAATCTTCATTGGTGGCTAAGGATGTGT

In [5]:
# read the sequence from a FASTA file LR782544.fasta
def extract_Arabidopsisthalianagenomeassemblysequence_from_fasta(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        sequence = ''.join(line.strip() for line in lines if not line.startswith('>'))
    return sequence

# Get the second sequence
file_path = "LR782544.fasta"
seq2 = extract_Arabidopsisthalianagenomeassemblysequence_from_fasta(file_path)
print("LR782544 is Sequence:", seq2)

print("The length of the LR782544 Sequence is:", len(seq2))

LR782544 is Sequence: GAGAATATTACACTCTTAAACACTTTTTTAAAACTATTTCTGTGTTTAGGCACTTTCAAAAGATTAGGCACTTTTTTATGTTTTTTTACTAAAATACCCTCTTAACTAAAATATTATCTTAACCCACGTTTTCTCTCTTTCTCTCTCTCTCTTCATCTCTCTCTTTCTCTCTCTCTCTCTCTTCATCTCTCTCTCTCTCTTCACCTCTCTCTCTCTTCATCTTTGTAAAAAATGAGAAGAAACTTCTTCTTCTTCAACAATGGCGGTTTGATTTGATTTTTGCTCTCTCTCATGTGTCAAACTCGTTTCATCTACGTAGCATCGAGATTTCAGTTAATCGAATCTCCTGTTATCATGGACATGTTATATAAAATGGATATAGATCAGTAAAAAGGTTTCCGAATGGATATATTTAAATGGATTCGTTATTATGGACACCAAAAAATTGTTTGTTGATTTTTTTTTCAGTTTGCAGAGACCTCCACCTTCAAAACCTTATGGACATGGAACTTTGGTTTCTTTACAAAACACGGTTAATCGAATCTCCTATTATTATTGACATGGTATATAGAATGGATACGGATCAGTAGAAAGGTTTCTGAATGGATATATTTAAATGGATTTGTTATTATGGATACCAAAAAATTATTTGTAGCAAATCAACCGCATATAGCTTGTTTTTAGGATGTTAGACATGCTTCTCCAATTCTTCATCTTTTACTCTTAACATGTAAGACCAAAAGATTTACAAATCTCACCATATCAATCCATAATATACAATCCATGATGCGTCATCCATAATATGTCATCCATGATTCATCATCCATAATGTGCCATCCATAATTTGCCATCCATAATGGACATCACCAATAACAAACGCCGTAAGAGCTACAAACACATCCTTAGCCACCAATGAAGATTTACCAAACACAAATAATTCCAATGAACAAGATTTCAACAAGAACACAAATTCTTTTT

In [6]:
matrix_values = open("blosum_example.txt", 'r')
substitution_matrix = {}

# Split the values into lines
lines = matrix_values.read().strip().split('\n')

# Iterate over lines to build the dictionary to store values in Key:Value format
for i in range(1, len(lines)):
    row_values = lines[i].split()
    Nucleic_acid1 = row_values[0]
    for j in range(1, len(row_values)):
        Nucleic_acid2 = lines[j][0]  # This should be corrected to row_values[j][0]
        score = int(row_values[j])
        substitution_matrix[(Nucleic_acid1, Nucleic_acid2)] = score

matrix_values.close()

In [7]:
# Initialise the variables
gap_penalty = -2
len_seq1 = len(seq1)
len_seq2 = len(seq2)
score_matrix = []

# Initialize the matrix with zeros
for i in range(len_seq1 + 1):
    row = [0] * (len_seq2 + 1)
    score_matrix.append(row)

# Fill first row and first column to be 0.
for i in range(1, len_seq1 + 1):
    score_matrix[i][0] = i * 0
for j in range(1, len_seq2 + 1):
    score_matrix[0][j] = j * 0

In [8]:
# Initialize a counter for printing the error message
error_count = 0
# Fill the matrix with scores based on match, mismatch, and gap penalties
for i in range(1, len_seq1 + 1):
    for j in range(1, len_seq2 + 1):
        Nucleic_acid1 = seq1[i - 1]
        Nucleic_acid2 = seq2[j - 1]
        if (Nucleic_acid1, Nucleic_acid2) in substitution_matrix:
            match = score_matrix[i-1][j-1] + substitution_matrix[(Nucleic_acid1, Nucleic_acid2)]
            delete = score_matrix[i-1][j] + gap_penalty
            insert = score_matrix[i][j-1] + gap_penalty
            score_matrix[i][j] = max(0, match, delete, insert)
        else:
            # Print the error message up to 2 of times
            if error_count < 2: 
                print("Pair of Nucleotide " + Nucleic_acid1 + Nucleic_acid2 + " doesn't have a corresponding entry in your substitution matrix.")
                error_count += 1

Pair of Nucleotide AG doesn't have a corresponding entry in your substitution matrix.
Pair of Nucleotide AG doesn't have a corresponding entry in your substitution matrix.


In [10]:
# Find the row index and column index of the maximum value
max_value = max(max(row) for row in score_matrix)

# Initialize variables to store indices
row_index = None
col_index = None

# Iterate over rows in reverse order
for i in range(len(score_matrix) - 1, -1, -1):
    # Iterate over columns in reverse order
    for j in range(len(score_matrix[i]) - 1, -1, -1):
        # Check if the current cell contains the maximum value
        if score_matrix[i][j] == max_value:
            row_index = i
            col_index = j
            # Break out of the inner loop as we found the first occurrence
            break
    # Break out of the outer loop if we found the first occurrence
    if row_index is not None:
        break

# Output the results
print("Maximum Value in the Alignment matrix:", max_value)
print("Row Index of Maximum Value:", row_index)
print("Column Index of Maximum Value:", col_index)

Maximum Value in the Alignment matrix: 3067
Row Index of Maximum Value: 1871
Column Index of Maximum Value: 1733


In [11]:
# Initialize variables for traceback
aligned_seq1 = []
aligned_seq2 = []
i, j = row_index, col_index  # Start from the maximum value's position

while i > 0 and j > 0:
    try:
        if score_matrix[i][j] == score_matrix[i-1][j-1] + substitution_matrix[(seq1[i-1], seq2[j-1])]:
            aligned_seq1.insert(0, seq1[i-1])
            aligned_seq2.insert(0, seq2[j-1])
            i -= 1
            j -= 1
        elif score_matrix[i][j] == score_matrix[i-1][j] + gap_penalty:
            aligned_seq1.insert(0, seq1[i-1])
            aligned_seq2.insert(0, '-')
            i -= 1
        else:
            aligned_seq1.insert(0, '-')
            aligned_seq2.insert(0, seq2[j-1])
            j -= 1
     
    except KeyError:
        # Handle KeyError, for example, by skipping the current iteration
        i -= 1
        j -= 1

aligned_seq1 = ''.join(aligned_seq1)
aligned_seq2 = ''.join(aligned_seq2)

print("Aligned Sequence 1:", aligned_seq1)
print("Aligned Sequence 2:", aligned_seq2)

Aligned Sequence 1: AATA-TATA-TCTTA-ACCAAACA--ATTAAAAAAATAAAAG--AATTTAG-ATACG-T-A---G-TTATTAATATAG--ACCA---TTA-GATTGAAAAATAAAAATTAAGATCTATGGCTGAGATTAAAGACAATA-AATGGATTAATTTTTTG----ATGTTAAAATCTGATTAGAAAAAGGTATTTCTCTTCGTCTCTAAAACTAAATCTCTCTCTCTAAAAAAACAATCGTTTCTCCCTTTCTCCTTCCTGAAGATCGT-TTTTTCATAAATCCATAGTA------G-------------T-TT-AAAAACGAAG-CAG---AGA---GA-TGTTGAAAATCGTTTCTCATG--AAATTAATCG-ATTATTCTCTGTGAAGTTCTTTAATCCACACAACTTTCCTCATG----AACATG--ATAATAGT-AGTAAATGGAGGTTTTTCCTATGGTTA-CTCTA-G-ACG----------AA-GG--A-GGATCTCCTTG-TG------TTGGACAG----G-----TTTG--TG-ATTTCTTTCCATG--------GATTA--AAAAAATTTGATTGTTTG-TTTATGATGAACGATTCTTTG----GCTACGGAAGAGTGTCATGGAGTTCTGGCG-AATTCTTTG--G-CTATG----TTTGGTGATTTCGTTTTTA----ATCAAG-TTGGGAATCAATAGGAAACAACTA-----A--G-CATACAACATAG----ATTAGAAGAG-ATATCAAGATGGATCTAATTTAAGTAAG-----ATTTGGCGACTAATTCTAG-ATG-----A-TTA---GG----GTTATTTGTG-ATTTATTACAAGGCATTTGTGTTCTCA-----TTGATTTG--GCGAGT-AATTCTG--TATG-ACTAGGGTTATTTG---TG-----TTTTCTTAAAAAGAATTTGTGTTCTTGTTGAAATCTTGTTCATTGGAATTA

In [15]:
# Write the Local alignment to a text file
sw_file_path = "Local_alignment_results.txt"
with open(sw_file_path, 'w') as sw_alignment:
    sw_alignment.write("Sequence 1: " + aligned_seq1 + "\n")
    sw_alignment.write("Sequence 2: " + aligned_seq2 + "\n")

print("Local Alignment results written to", sw_file_path)

Local Alignment results written to Local_alignment_results.txt


In [14]:
# Specify the file path to store the store matrix
file_path = "SW_score_matrix.txt"

# Save the matrix to a text file
with open(file_path, "w") as file:
    for row in score_matrix:
        file.write(" ".join(map(str, row)) + "\n")

print(f"Score matrix for Local alignment has been saved to {file_path}")

Score matrix for Local alignment has been saved to SW_score_matrix.txt
